# **Chains**

chains are a fundamental concept that allows you to execute complex tasks in a structured and efficient way.

## **Why Chains?**

Chains are invaluable due to their capacity to effortlessly blend diverse components, shaping a singular and coherent application. Through the creation of chains, multiple elements can seamlessly come together. Imagine this scenario: a chain is crafted to take in user input, polish it using a PromptTemplate, and subsequently pass on this refined response to a large language model (LLM). This streamlined process not only simplifies but also enriches the overall functionality of the system. In essence, chains serve as the linchpin, seamlessly connecting different parts of the application and enhancing its capabilities.


In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain langchain_community langchain-openai
!pip install pandas
!pip install --upgrade python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["OPENAI_API_VERSION"] = os.getenv('OPENAI_API_VERSION')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv('AZURE_OPENAI_API_KEY')

In [6]:
import pandas as pd
df = pd.read_csv("/content/employee.csv")

In [7]:
df.head()

,Employee ID,Employee Name,Designation,Tools Used,Date of Birth,Salary,Hire Date,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,1,John Doe,Software Engineer,"Eclipse, Git, JIRA",15-03-1985,"$75,000",20-06-2010,NaN,NaN,NaN,NaN,NaN
1,2,Jane Smith,UI/UX Designer,"Figma, Adobe XD, Sketch",22-08-1990,"$55,000",10-02-2019,NaN,NaN,NaN,NaN,NaN
2,3,Alice Brown,Database Administrator,"MySQL, MongoDB, Oracle",10-11-1988,"$60,000",05-04-2015,NaN,NaN,NaN,NaN,NaN
3,4,Bob White,DevOps Engineer,"Jenkins, Docker, Kubernetes",02-04-1980,"$80,000",15-09-2013,NaN,NaN,NaN,NaN,NaN
4,5,Charlie Black,QA Engineer,"Selenium, JUnit, TestRail",18-07-1992,"$65,000",30-10-2020,NaN,NaN,NaN,NaN,NaN


# **LLM Chain** - **The simplest chain**

The LLMChain is a foundational system that includes a PromptTemplate, an OpenAI model (such as a Large Language Model or a ChatModel), and optionally, an output parser. It operates by transforming input parameters using the PromptTemplate into a coherent prompt, which is then fed into the model. The resulting output is further refined and formatted into a usable form by the OutputParser, if provided. This structured approach ensures effective utilization of language models for various applications, enhancing their functionality and utility.

In [27]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [28]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
    temperature = 0.5
)

In [30]:
prompt = ChatPromptTemplate.from_template(
    "What are the tools that need to be learned to earn the {designation}?,provide me one tool"
)

In [31]:
chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-31-1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [32]:
designation = "Devops Engineer"
chain.run(designation)

/tmp/ipython-input-32-4066474957.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(designation)


'To become a DevOps Engineer, you need to be familiar with a variety of tools that span different stages of the software development lifecycle, including development, version control, CI/CD pipelines, configuration management, containerization, orchestration, monitoring, and more.\n\nHere is **one essential tool** to get started: \n\n### **Git (Version Control Tool)**\n\n- **What is Git?**\n  Git is a distributed version control system that allows developers to track changes in source code during software development. It enables collaboration among team members by allowing them to work on the same codebase simultaneously.\n\n- **Why is Git important for DevOps?**\n  Git is foundational in DevOps because it facilitates collaboration, version tracking, and integration with CI/CD pipelines. Almost all DevOps workflows start with source code management in Git.\n\n- **Key Concepts to Learn:**\n  - Repositories (local and remote)\n  - Branching and merging\n  - Pull requests\n  - Tags\n  - R

# **SimpleSequentialChain**

Simple Sequential Chains allow for a single input to undergo a series of coherent transformations, resulting in a refined output. This sequential approach ensures systematic and efficient handling of data, making it ideal for scenarios where a linear flow of information processing is essential

In [35]:
# SimpleSequentialChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [36]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What are the tools that need to be learned to earn the {designation}?,provide me one tool"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [37]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    tool:{tool_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [38]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [39]:
overall_simple_chain.run(designation)



> Entering new SimpleSequentialChain chain...
To become a DevOps Engineer, you need to learn a variety of tools that are used for automation, continuous integration/continuous delivery (CI/CD), containerization, orchestration, monitoring, and more. These tools help streamline the development and operations processes, enabling faster and more reliable software delivery.

One key tool you should learn is **Docker**.

### About Docker:
Docker is a popular containerization platform that allows you to create, deploy, and run applications in lightweight, portable containers. Containers package an application and all its dependencies together, ensuring that it runs consistently across different environments.

#### Why Docker is important for DevOps Engineers:
1. **Consistency Across Environments**: Docker ensures that applications behave the same way in development, testing, and production environments.
2. **Efficient Resource Utilization**: Containers are lightweight compared to virtual ma

'Docker is a leading containerization tool enabling consistent, lightweight application deployment across environments. Essential for DevOps, it streamlines scalability, efficiency, and CI/CD workflows.'

# **SequentialChain**

A sequential chain is a chain that combines various individual chains, where the output of one chain serves as the input for the next in a continuous sequence. It operates by running a series of chains consecutively.

In [40]:
# SequentialChain
from langchain.chains import SequentialChain

In [41]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following designation"
    "\n\n{Designation}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Designation_description"
                    )


In [42]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following description in 1 sentence:"
    "\n\n{Designation_description}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [43]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What are the promgramming language that need to be learned for this designation in one line:\n\n{Designation}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="programming_language"
                      )

In [44]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a short follow up response to the following "
    "Write a short summary about the programming language:"
    "\n\nSummary: {summary}\n\nLanguage: {programming_language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [45]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Designation"],
    output_variables=["Designation_description","programming_language", "summary","followup_message"],
    verbose=True
)

In [46]:
designation = df.Designation[2]
overall_chain(designation)

/tmp/ipython-input-46-3036767809.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(designation)




> Entering new SequentialChain chain...

> Finished chain.


{'Designation': 'Database Administrator',
 'Designation_description': 'A Database Administrator manages, secures, and optimizes databases, ensuring data integrity, availability, and performance while supporting organizational data needs.',
 'programming_language': 'For a Database Administrator, you should learn **SQL**, **PL/SQL** (or equivalent procedural extensions), **T-SQL**, and optionally scripting/programming languages like **Python**, **Bash**, or **PowerShell** for automation and database management tasks.',
 'summary': 'A Database Administrator ensures databases are secure, efficient, and reliable to meet organizational data requirements.',
 'followup_message': "Follow-up: In addition to the core languages like SQL, PL/SQL, and T-SQL, it's beneficial for a Database Administrator to familiarize themselves with database-specific tools and platforms (e.g., Oracle, Microsoft SQL Server, MySQL, or PostgreSQL). Understanding version control systems (e.g., Git) and cloud database se

# **Router Chain**

The Router Chain is used for complicated tasks. If we have multiple subchains, each of which is specialized for a particular type of input, we could have a router chain that decides which subchain to pass the input to.

In [47]:
software_engineer_template = """You are a highly skilled software engineer. \
You excel at addressing queries about programming and software development in a clear
and straightforward manner. \
When faced with a challenge outside your expertise, you candidly \
acknowledge the gap in your knowledge. \

Here is a question:
{input}"""


database_administrator_template = """You are a highly skilled database administrator.\
You excel at answering questions about databases in a clear and precise manner, \
making even complex topics accessible to anyone.\
When faced with a query or issue you're unfamiliar with, \
you candidly admit that you need to look into it further, valuing accuracy over conjecture.\

Here is a question:
{input}"""

data_scientist_template = """You are a highly skilled data scientist.\
You excel at interpreting complex datasets and presenting insights in a \
straightforward and comprehensible manner.\
When confronted with a query beyond your expertise, you acknowledge \
your limitations and advocate for further research or consultation.

Here is a question:
{input}"""


Machine_learning_engineer_template = """ You are a highly skilled Machine Learning Engineer.\
You excel at answering questions about machine learning algorithms and models in a clear and succinct manner.\
When confronted with a query outside your expertise, you candidly admit that you don't have the answer. \

Here is a question:
{input}"""

In [48]:
prompt_infos = [
    {
        "name": "software_engineer",
        "description": "Good for answering questions about software_engineer",
        "prompt_template": software_engineer_template
    },
    {
        "name": "database_administrator",
        "description": "Good for answering database_administrator questions",
        "prompt_template": database_administrator_template
    },
    {
        "name": "data_scientist",
        "description": "Good for answering data_scientist questions",
        "prompt_template": data_scientist_template
    },
    {
        "name": "Machine_learning_engineer",
        "description": "Good for answering Machine_learning_engineer questions",
        "prompt_template": Machine_learning_engineer_template
    }
]

In [53]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [54]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [55]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [56]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [57]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [58]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-58-2333002322.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [59]:
chain.run("What are the various categories of software?")



> Entering new MultiPromptChain chain...
software_engineer: {'input': 'What are the various categories of software?'}
> Finished chain.


'Software can be categorized into various types based on its purpose, functionality, and usage. Below are the primary categories of software:\n\n---\n\n### 1. **System Software**\n   - **Definition**: Software that provides a platform for other software and manages the hardware of a computer.\n   - **Examples**:\n     - Operating Systems (e.g., Windows, macOS, Linux)\n     - Utility Programs (e.g., disk cleanup tools, antivirus software)\n     - Device Drivers (e.g., printer drivers, GPU drivers)\n   - **Purpose**: To manage and control hardware resources and provide essential services for application software.\n\n---\n\n### 2. **Application Software**\n   - **Definition**: Software designed to perform specific tasks for users.\n   - **Examples**:\n     - Productivity Tools (e.g., Microsoft Word, Excel, Google Docs)\n     - Web Browsers (e.g., Chrome, Firefox, Safari)\n     - Media Players (e.g., VLC, Spotify)\n     - Games\n   - **Purpose**: To help users accomplish specific tasks or 

In [60]:
chain.run("What are the differences between supervised and unsupervised learning")



> Entering new MultiPromptChain chain...
Machine_learning_engineer: {'input': 'What are the differences between supervised and unsupervised learning?'}
> Finished chain.


'Supervised and unsupervised learning are two primary types of machine learning paradigms, and they differ in several key ways:\n\n### 1. **Definition:**\n   - **Supervised Learning:** In supervised learning, the model is trained on labeled data, meaning the input data comes with corresponding output labels. The goal is to learn a mapping from inputs to outputs.\n   - **Unsupervised Learning:** In unsupervised learning, the model is trained on unlabeled data. The goal is to identify patterns, structures, or relationships in the data without explicit output labels.\n\n---\n\n### 2. **Data:**\n   - **Supervised Learning:** Requires labeled data, where each input has a known output (e.g., images with labels like "cat" or "dog").\n   - **Unsupervised Learning:** Works with unlabeled data, where no predefined labels or outputs are provided.\n\n---\n\n### 3. **Objective:**\n   - **Supervised Learning:** Predict outcomes for new, unseen data based on the labeled training data. Common tasks ar

# **Let's Do an Activity**

## **Objective**

Practice using chains with language models and structured output parsing to handle complex tasks efficiently.

## **Scenario**

You are tasked with building a system to assist users in understanding various technical concepts related to software engineering roles. Your goal is to use different chains to process user queries, transform them using prompt templates, and extract specific information using output parsers.

## **Steps**

* Define a Prompt Template
* Prepare Sample Queries
* Implement LLM Chains
* Output Parsing
* Interactive Chain Execution
* Evaluate